In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('weather_clean.csv', index_col=0)
data

,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,...,DR,SH,FZ,MI,PR,BC,BL,VC,units,item_nbr
0,48,33,41,16,37,39,24,0,716,1626,...,False,False,False,False,False,False,False,False,0,1
1,48,33,41,16,37,39,24,0,716,1626,...,False,False,False,False,False,False,False,False,0,2
2,48,33,41,16,37,39,24,0,716,1626,...,False,False,False,False,False,False,False,False,0,3
3,48,33,41,16,37,39,24,0,716,1626,...,False,False,False,False,False,False,False,False,0,4
4,48,33,41,16,37,39,24,0,716,1626,...,False,False,False,False,False,False,False,False,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1510483,45,33,39,-6,24,32,26,0,624,1646,...,False,False,False,False,False,False,False,False,0,107
1510484,45,33,39,-6,24,32,26,0,624,1646,...,False,False,False,False,False,False,False,False,0,108
1510485,45,33,39,-6,24,32,26,0,624,1646,...,False,False,False,False,False,False,False,False,0,109
1510486,45,33,39,-6,24,32,26,0,624,1646,...,False,False,False,False,False,False,False,False,0,110


In [3]:
# В рамках протоптипа будут рассмотрено прогнозирование товаров в городе Dallas
dallas = data[data.city == 'Dallas']
dallas 

,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,...,DR,SH,FZ,MI,PR,BC,BL,VC,units,item_nbr
444,63,47,55,4,28,43,10,0,728,1742,...,False,False,False,False,False,False,False,False,0,1
445,63,47,55,4,28,43,10,0,728,1742,...,False,False,False,False,False,False,False,False,0,2
446,63,47,55,4,28,43,10,0,728,1742,...,False,False,False,False,False,False,False,False,0,3
447,63,47,55,4,28,43,10,0,728,1742,...,False,False,False,False,False,False,False,False,0,4
448,63,47,55,4,28,43,10,0,728,1742,...,False,False,False,False,False,False,False,False,38,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509262,76,56,66,-1,47,56,0,1,644,1745,...,False,False,False,False,False,False,False,False,0,107
1509263,76,56,66,-1,47,56,0,1,644,1745,...,False,False,False,False,False,False,False,False,0,108
1509264,76,56,66,-1,47,56,0,1,644,1745,...,False,False,False,False,False,False,False,False,0,109
1509265,76,56,66,-1,47,56,0,1,644,1745,...,False,False,False,False,False,False,False,False,0,110


* Добавить GridSearch. <br>
* Перебрать lam, n_splines. <br>
* Попробовать внедрить тензорный сплайн (from pygam import te) для рассмотрения влияния типов погоды ([17:]) на основные признаки. <br>

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pygam import PoissonGAM
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

# список коэффициентов Dallas
coefs = []

dallas[['+FC', 'FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG+', 'FG', 'BR', 'UP', \
               'HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', \
                'BC', 'BL', 'VC']] = \
        dallas[['+FC', 'FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG+', 'FG', 'BR', 'UP', \
               'HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', \
                'BC', 'BL', 'VC']].astype('int')

dallas = dallas.drop('city', axis=1)

# Функция обучения на один тип товара и получения коэффициентов для этого товара
def pipeline(df, i):
    
    y = df["units"]
    X = df.drop(columns=["units", 'item_nbr'])

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    gam = PoissonGAM(max_iter=100000).fit(X_train, y_train.to_numpy())
    y_pred = gam.predict(X_test)

    # Вывод MAE для отслеживания прогресса
    print(f" №{i}: {round(mean_absolute_error(y_test, y_pred), 2)}")
    return gam.coef_

# Перебор всех типов товаров
for i in range(1, 112):
    _df = dallas[dallas['item_nbr'] == i]
    if _df.units.unique().shape[0] == 1:
        coefs.append(np.array([0 for _ in range(1001)]))
    else:
        coefs.append(pipeline(_df, i))


        
# Формула расчета через коэфы:
#B = gam._modelmat(X_test[i].reshape(1, -1))
#pred = round(np.exp(float(B @ coefficients)))

 №5: 26.24
 №23: 4.57
 №59: 0.87
 №84: 0.19
 №93: 1.81
 №100: 0.86


In [6]:
coefs

[array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([-0.51017116,  0.2468735 ,  0.54498265, ..., -0.07154241,
        -0.05395551,  0.20495232]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([ 0.06567197, -0.21466307, -0.08034711, ..., -0.12085744,
        -0.10931546,  0.15892226]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array

In [7]:
coefficents = pd.DataFrame({'Dallas': coefs}, index=[i for i in range(1, 112)])
coefficents

,Dallas
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,"[-0.5101711574227303, 0.2468735006857925, 0.54..."
...,...
107,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
108,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
109,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
110,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [8]:
coefficents.to_csv('weather_coefs_OLD.csv')